# Notebook Overview
1. Took the summary of AI
2. Preprocessed, Tokenization, padding, Split the data, and train the baseline LSTM model
3. Implelemted the hyperparameter tuning with the Random search and Bayesian Optimization
4. Overfitting was tackled by implementing dropouts, batch normalization and so on
5. Evalauted the model with the metrics Perplexity, Blue score and Rouge Score.

Final Outcome: Since data was less and the model was not able to generalise more.


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=18cb2197625667b407de20254978a8872f7e1fcd6f28c27a2d688533a44c9803
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [3]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [4]:
import re
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import keras_tuner as kt


In [5]:

# Sample text corpus
text = """**Artificial Intelligence: Transforming the Future**

### Introduction

Artificial Intelligence (AI) is one of the most revolutionary advancements of the 21st century. It has permeated every aspect of human life, from healthcare and finance to transportation and entertainment. AI refers to the simulation of human intelligence in machines, enabling them to learn, reason, and perform tasks that traditionally required human cognition. As AI continues to evolve, it raises critical questions about its potential, ethical implications, and societal impact.

### The Evolution of AI

The concept of AI dates back to ancient mythology, where automated beings were imagined. However, AI as a scientific discipline emerged in the mid-20th century. Alan Turing, a pioneer in computing, proposed the idea of machines that could mimic human intelligence. In 1956, the Dartmouth Conference officially introduced AI as a field of study.

AI development can be categorized into three waves:

1. **Symbolic AI (1950s–1980s)**: Early AI relied on rule-based systems and logic programming. However, these systems struggled with complex tasks requiring intuition or vast amounts of data.
2. **Machine Learning (1990s–2010s)**: The advent of neural networks and statistical models enabled machines to learn from data rather than relying solely on pre-defined rules.
3. **Deep Learning and AI Today (2010s–present)**: Advances in computing power and data availability have led to deep learning models that surpass human performance in specific tasks, such as image recognition and natural language processing.

### Types of Artificial Intelligence

AI is broadly categorized into three types:

1. **Narrow AI (Weak AI)**: This type of AI is designed for specific tasks, such as facial recognition, recommendation systems, and autonomous vehicles. Examples include Siri, Google Assistant, and Netflix’s recommendation engine.
2. **General AI (Strong AI)**: General AI possesses human-like cognitive abilities, enabling it to understand, learn, and apply knowledge across different domains. This level of AI remains theoretical.
3. **Super AI**: A hypothetical AI surpassing human intelligence, potentially possessing self-awareness, reasoning abilities, and superior problem-solving skills. This concept is often discussed in science fiction and philosophical debates.

### Applications of AI

AI has penetrated various industries, revolutionizing traditional processes and enhancing efficiency. Some key applications include:

#### 1. **Healthcare**
AI-powered systems assist in diagnosing diseases, predicting patient outcomes, and personalizing treatment plans. For example, IBM Watson can analyze medical literature and provide insights to doctors. AI is also instrumental in drug discovery and robotic-assisted surgeries.

#### 2. **Finance**
In the financial sector, AI-driven algorithms detect fraudulent transactions, automate trading strategies, and provide personalized financial advice. AI chatbots enhance customer service by resolving queries in real time.

#### 3. **Transportation**
AI is the backbone of autonomous vehicles, optimizing traffic management and enhancing safety. Companies like Tesla and Waymo are pioneering self-driving car technology.

#### 4. **Retail and E-commerce**
AI personalizes customer experiences by analyzing purchase behavior and recommending products. Chatbots and virtual assistants streamline customer service, while AI-driven supply chain management improves logistics.

#### 5. **Entertainment and Media**
Streaming services like Netflix and Spotify use AI to recommend content based on user preferences. AI also generates music, scripts, and digital art, transforming the creative industry.

### Ethical and Social Implications

While AI brings numerous benefits, it also raises ethical concerns:

#### 1. **Job Displacement**
Automation threatens traditional jobs, particularly in industries like manufacturing and customer service. However, it also creates new opportunities in AI development, data science, and cybersecurity.

#### 2. **Bias and Fairness**
AI models can inherit biases from training data, leading to unfair outcomes in hiring, lending, and law enforcement. Ensuring transparency and fairness in AI decision-making is crucial.

#### 3. **Privacy Concerns**
AI-driven surveillance and data collection pose privacy risks. Governments and organizations must establish robust regulations to protect user data.

#### 4. **Autonomous Weapons**
The development of AI-powered military systems raises concerns about ethical warfare. Autonomous weapons may operate without human intervention, leading to unintended consequences.

### The Future of AI

The future of AI holds immense potential. Researchers are working on enhancing AI’s interpretability, ensuring ethical AI practices, and developing human-AI collaboration models. Some emerging trends include:

1. **Explainable AI (XAI)**: Efforts are being made to create transparent AI systems that provide reasoning behind their decisions.
2. **AI in Space Exploration**: AI is being used to analyze cosmic data, predict astronomical events, and assist in interplanetary missions.
3. **AI for Climate Change**: AI models predict climate patterns, optimize energy consumption, and enhance sustainability efforts.
4. **AI and Human Enhancement**: Brain-computer interfaces and AI-driven prosthetics are revolutionizing healthcare and augmenting human capabilities.

### Conclusion

Artificial Intelligence is transforming the world in unprecedented ways. While it presents remarkable opportunities, ethical challenges must be addressed to ensure responsible AI deployment. As AI continues to evolve, society must strike a balance between innovation and ethical considerations, fostering a future where AI benefits humanity without compromising fundamental values.


"""


# Download nltk resources
#nltk.download('stopwords')
nltk.download('wordnet')

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'@\w+|#\w+|\*+', '', text)  # Remove @, #, *
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = text.split()

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

# Apply preprocessing
clean_text = preprocess_text(text)


print(clean_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...


artificial intelligence transforming the future introduction artificial intelligence ai is one of the most revolutionary advancement of the 21st century it ha permeated every aspect of human life from healthcare and finance to transportation and entertainment ai refers to the simulation of human intelligence in machine enabling them to learn reason and perform task that traditionally required human cognition a ai continues to evolve it raise critical question about it potential ethical implication and societal impact the evolution of ai the concept of ai date back to ancient mythology where automated being were imagined however ai a a scientific discipline emerged in the mid20th century alan turing a pioneer in computing proposed the idea of machine that could mimic human intelligence in 1956 the dartmouth conference officially introduced ai a a field of study ai development can be categorized into three wave 1 symbolic ai 1950s1980s early ai relied on rulebased system and logic progra

In [14]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([clean_text])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences
input_sequences = []
words = clean_text.split()

for i in range(1, len(words)):
    sequence = words[:i+1]  # Create n-gram sequences
    input_sequences.append(tokenizer.texts_to_sequences([' '.join(sequence)])[0])

# Pad sequences
max_seq_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre')

# Create input (X) and output (y)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# Split into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning function
def build_model(hp):
    model = Sequential([
        Embedding(input_dim=total_words, output_dim=hp.Int('embedding_dim', min_value=50, max_value=200, step=50), input_length=max_seq_length-1),
        LSTM(hp.Int('lstm_units', min_value=50, max_value=200, step=50), return_sequences=True),
        Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)),
        LSTM(hp.Int('lstm_units_2', min_value=50, max_value=200, step=50)),
        Dense(hp.Int('dense_units', min_value=50, max_value=200, step=50), activation='relu'),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Hyperparameter tuning with Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='text_gen_lstm'
)

# Run the tuner
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# Train the best model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1)

# Evaluation Metrics
def calculate_perplexity(model, X, y):
    loss, _ = model.evaluate(X, y, verbose=0)  # Categorical cross-entropy loss
    return np.exp(loss)

def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        tokenized_input = tokenizer.texts_to_sequences([seed_text])
        padded_input = pad_sequences(tokenized_input, maxlen=max_seq_length-1, padding='pre')
        predicted_word_index = np.argmax(model.predict(padded_input, verbose=0), axis=-1)[0]

        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                seed_text += " " + word
                break
    return seed_text



Reloading Tuner from hyperparam_tuning/text_gen_lstm/tuner0.json
Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.0393 - loss: 5.9893 - val_accuracy: 0.0452 - val_loss: 5.8096
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.0675 - loss: 5.5781 - val_accuracy: 0.0452 - val_loss: 6.0086
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.0665 - loss: 5.4523 - val_accuracy: 0.0452 - val_loss: 6.3045
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.0665 - loss: 5.4271 - val_accuracy: 0.0452 - val_loss: 6.4766
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.0610 - loss: 5.4293 - val_accuracy: 0.0452 - val_loss: 6.6061
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - accuracy: 0.0743 - loss: 5.3805 - val_accuracy: 0.0452 - val_loss: 6.6954
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.0741 - loss: 5.3828 - val_accuracy: 0.0452 - val_loss: 6.7348
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model with Regularization, Dropout, and Early Stopping
def build_model(hp):
    model = Sequential([
        Embedding(input_dim=total_words, output_dim=hp.Int('embedding_dim', min_value=50, max_value=200, step=50), input_length=max_seq_length-1),
        LSTM(hp.Int('lstm_units', min_value=50, max_value=100, step=50), return_sequences=True, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)),
        LSTM(hp.Int('lstm_units_2', min_value=50, max_value=100, step=50), kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dense(hp.Int('dense_units', min_value=50, max_value=100, step=50), activation='relu'),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
    return model

# Hyperparameter tuning
tuner_new = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='hyperparam_tuning1',
    project_name='text_gen_lstm1'
)

# Run the tuner
tuner_new.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# Get best model and train
best_hps = tuner_new.get_best_hyperparameters(num_trials=5)[0]
model = tuner_new.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1)

Trial 5 Complete [00h 02m 11s]
val_accuracy: 0.05161290243268013

Best val_accuracy So Far: 0.08387096971273422
Total elapsed time: 00h 12m 45s
Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.0048 - loss: 8.9676 - val_accuracy: 0.0000e+00 - val_loss: 8.8608
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.0209 - loss: 8.7422 - val_accuracy: 0.0000e+00 - val_loss: 8.7644
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.0258 - loss: 8.5820 - val_accuracy: 0.0000e+00 - val_loss: 8.6722
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.0357 - loss: 8.4331 - val_accuracy: 0.0000e+00 - val_loss: 8.5857
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.0488 - loss: 8.2755 - val_accuracy: 0.0000e+00 - val_loss: 8.5022
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.0603 - loss: 8.1180 - val_accuracy: 0.0000e+00 - val_loss: 8.4202
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 

In [23]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Function to build the model
def build_model(hp):
    model = Sequential([
        Embedding(input_dim=total_words,
                  output_dim=hp.Int('embedding_dim', min_value=50, max_value=200, step=50),
                  input_length=max_seq_length-1),
        LSTM(hp.Int('lstm_units', min_value=50, max_value=100, step=50),
             return_sequences=True, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)),
        LSTM(hp.Int('lstm_units_2', min_value=50, max_value=100, step=50),
             kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dense(hp.Int('dense_units', min_value=50, max_value=100, step=50), activation='relu'),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Use Bayesian Optimization instead of Random Search
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    executions_per_trial=1,
    directory='hyperparam_tuning5',
    project_name='text_gen_lstm_bayesian'
)

# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=30, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build and train the best model
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)


Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.03870967775583267

Best val_accuracy So Far: 0.06451612710952759
Total elapsed time: 00h 02m 54s
Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.0052 - loss: 8.7204 - val_accuracy: 0.0387 - val_loss: 8.0966
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.0403 - loss: 7.7695 - val_accuracy: 0.0452 - val_loss: 7.5115
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.0633 - loss: 6.9532 - val_accuracy: 0.0452 - val_loss: 7.1264
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.0971 - loss: 6.2769 - val_accuracy: 0.0452 - val_loss: 6.8911
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.0943 - loss: 5.7828 - val_accuracy: 0.0452 - val_loss: 6.7387
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.0770 - loss: 5.3173 - val_accuracy: 0.0452 - val_loss: 6.6379
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.1248 - loss: 4.9117 - 

In [24]:
# Example evaluation
generated_text = generate_text("Artificial Intelligence", next_words=30)

# Assuming reference_text is a list of lists
reference_text = [["Artificial Intelligence (AI) is one of the most revolutionary advancements of the 21st century"]]
candidate_text = generated_text.split()

# Compute BLEU Score
smooth_fn = SmoothingFunction().method1  # Apply smoothing
bleu_score = sentence_bleu(reference_text, candidate_text, smoothing_function=smooth_fn)

# Compute ROUGE Score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(" ".join(candidate_text), " ".join(reference_text[0]))

# Compute Perplexity
perplexity = calculate_perplexity(model, X_test, y_test)

# Print results
print(f"\nGenerated Text: {generated_text}")
print(f"BLEU Score: {bleu_score:.4f}")
print(f"ROUGE Scores: {rouge_scores}")
print(f"Perplexity: {perplexity:.4f}")


Generated Text: Artificial Intelligence intelligence transforming transforming intelligence future future intelligence intelligence future future artificial is is is is is ai the and a ai a in a a in a a ai a
BLEU Score: 0.0000
ROUGE Scores: {'rouge1': Score(precision=0.35714285714285715, recall=0.15625, fmeasure=0.21739130434782608), 'rouge2': Score(precision=0.07692307692307693, recall=0.03225806451612903, fmeasure=0.045454545454545456), 'rougeL': Score(precision=0.2857142857142857, recall=0.125, fmeasure=0.17391304347826086)}
Perplexity: 601.9516


In [12]:
# Evaluation Metrics
def calculate_perplexity(model, X, y):
    loss, _ = model.evaluate(X, y, verbose=0)  # Categorical cross-entropy loss
    return np.exp(loss)

def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        tokenized_input = tokenizer.texts_to_sequences([seed_text])
        padded_input = pad_sequences(tokenized_input, maxlen=max_seq_length-1, padding='pre')
        predicted_word_index = np.argmax(model.predict(padded_input, verbose=0), axis=-1)[0]

        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                seed_text += " " + word
                break
    return seed_text

# Example evaluation
generated_text = generate_text("Artificial ", next_words=30)

# Assuming reference_text is a list of lists
reference_text = [["Artificial Intelligence (AI) is one of the most revolutionary advancements in technology."]]
candidate_text = generated_text.split()

# Compute BLEU Score
smooth_fn = SmoothingFunction().method1  # Apply smoothing
bleu_score = sentence_bleu(reference_text, candidate_text, smoothing_function=smooth_fn)

# Compute ROUGE Score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(" ".join(candidate_text), " ".join(reference_text[0]))

# Compute Perplexity
perplexity = calculate_perplexity(model, X_test, y_test)

# Print results
print(f"\nGenerated Text: {generated_text}")
print(f"BLEU Score: {bleu_score:.4f}")
print(f"ROUGE Scores: {rouge_scores}")
print(f"Perplexity: {perplexity:.4f}")


Generated Text: Artificial  intelligence transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming transforming
BLEU Score: 0.0000
ROUGE Scores: {'rouge1': Score(precision=0.16666666666666666, recall=0.06451612903225806, fmeasure=0.09302325581395349), 'rouge2': Score(precision=0.09090909090909091, recall=0.03333333333333333, fmeasure=0.04878048780487805), 'rougeL': Score(precision=0.16666666666666666, recall=0.06451612903225806, fmeasure=0.09302325581395349)}
Perplexity: 293737530932.1476
